# Vectorized Entity Resolution: An Illustrative Case

In this excercise, we leverage a dataset of song information to see if we successfully resolve the number of entities into a true list of unique songs.  This dataset has already been resolved and the unique id for the true record is the column referred to as "CID."  You can find the entire data dictionary below (listed by column). Source: University of Leipzig.

## Step 1: Package Importation

As shown below, we will only be using packages that are part of the Anaconda Python suite.  

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, pairwise_kernels
from scipy import sparse
import numpy as np
from sklearn.preprocessing import normalize
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Step 2: Data Pre-Processing

The pre-processing required for this excercise is extremely straightforward.  In the first  and second cell, we import the dataframe and combine all of the identifying information (e.g. the song title and artist) into one cell.  From that point on, we will only use that column (called "all") and the cluster ID column to compare the records.

In [12]:
data = pd.read_csv(r'Albums.csv')
data.head()
#IF YOUR COMPUTER CANNOT RUN THE BELOW CODE BECAUSE OF A MEMORY ERROR, UNCOMMENT THE LINE BELOW
data = data[data['CID']>=500]

In [13]:
#It is critical to format each cell as a string before concatenating them.
data['all'] = data['title'].astype(str) + data['length'].astype(str) + data['artist'].astype(str) + data['album'].astype(str) + data['language'].astype(str)

In [14]:
data[data['CID']==6][['title', 'length', 'artist', 'album', 'year', 'language']]

,title,length,artist,album,year,language


## Step 3: Vectorization

This is the most important stage of the entity resolution process as the speed and efficiency of this step is what makes the entire process so useful.

In the cell immediately below, we first instantiate a vectorizer with a term-frequency-inverse-document-frequency (TFIDF) normalizer.  TFIDF will count the attributes (in this case combinations of characters) in each cell and develop a score of that for each one of those attributes that weighes rarer combinations of characters more than more common combinations.  For instance the character combination "xyz" will be wieghed more than the character combination "the".  For information on the TFIDF algorythim, please see here: .

The other important parameter of the vectorizer is the n_gram range, i.e. range of character combinations we select. In this case, we select 4 which means we will select character combinations of between 1 and 4 characters long.

After instantiating the vectorizer, we then fit the vectorizer to the "all" column and then transform a matrix that scores the entire dataset on those 1-4 combinations of characters on the TFIDF alogrythim.  From this process we yield a extremely large matrix that has the length of the number of records in the dataset and width of the number of all combinations of 1-4 characters in the dataset. 

In [15]:
#Instantiate the vector with the aforementioned parameteres
vect = TfidfVectorizer(analyzer='char', ngram_range=(1,4))
#Fit the vector and transform it into a scored matrix
matrix = vect.fit_transform(data['all'])

In the below cell, we compute a similarity score of the matrix using the cosine similarity function.  Cosine similarity is a measure of euclidean distance.  For more infomation about the cosine similarity function, please go here: .

From that function, we obtain a matrix that score each record on its similarity with every other record.  Output of the cosine similarity function has the dimensions of NxN  where N is the total number of records.  On the diagonals of the matrix is score of the records similarity with itself.  We zero out the diagonal scores so that we can find matches based off of a score threshold later in this process.

In [16]:
#Compute the cosine similarity matrix. You must produce a sparse output in order save memory
cos_sim = cosine_similarity(matrix, dense_output=False)
#Zero out diagonals of the cosine similarity matrix
cos_sim.setdiag(0)

MemoryError: Unable to allocate 1.26 GiB for an array with shape (337750876,) and data type int32

We now have all of the information that we need to resolve the song names in this dataset.  From the dataset documentation, we know that there are around 9000 duplicate names so we should determine the minimum cosine similarity score by looking at the portion of true duplicate records that we identify and the false positives we find.  

First however, let's take a look what happens when we set a filter parameter of .95.  In this case, we only find 250 matches (keep in mind that we there is a low chance of duplicated matches).  Using the argwhere function, we find the index values for the record matches and put that list into a dataframe (for ease of analyis). 

In [ ]:
#Filter matches to the indices where the score value is higher than .95
matches = np.argwhere(cos_sim>.95)
#Format into a dataframe for ease of use.
match_df= pd.DataFrame(matches)

In [ ]:
match_df.head()

In [ ]:
#Let's look at an example of a match (record 1)
data.iloc[305]

TID                                                       306
CID                                                       162
CTID                                                        1
SourceID                                                    5
id                                                   16288675
number                                                     E4
title               A Wand'ring Minstrel I, From "The Mikado"
length                                                 261000
artist              Sir William Gilbert & Sir Arthur Sullivan
album                         Golden Sounds From the Classics
year                                                      NaN
language                                              English
all         A Wand'ring Minstrel I, From "The Mikado"26100...
Name: 305, dtype: object

In [ ]:
#Here's the record match
data.iloc[15920]

TID                                                     15921
CID                                                       162
CTID                                                        3
SourceID                                                    2
id                                            MBox44023429-HH
number                                                     E4
title       Sir William Gilbert & Sir Arthur Sullivan - A ...
length                                                    261
artist                                                    NaN
album                         Golden Sounds From the Classics
year                                                      NaN
language                                              English
all         Sir William Gilbert & Sir Arthur Sullivan - A ...
Name: 15920, dtype: object

## Step 4: Analysis of Results

With an idea what a match looks like, how do we determine where we set the threshold for the cosine similarity function?  One way is to look at the tradeoff between true and false positives at varying levels of similarity.  The below cell compiles the number of correctly identified duplicate songs and incorrectly indentified songs at similaritys scores between .4 and 1 while filtering out duplicate matches (see code below).  This information is then compiled in a dataframe that we will analyze in the following cells.

In [ ]:
analysis_df = pd.DataFrame()
for i in range(40,100,2):
    #Iterate through several threshold scores.
    threshold_val = i/100
    matches = np.argwhere(cos_sim>threshold_val)
    
    match_count = 0
    error_count= 0
    errors_found=[]
    dupes_found =[]
    
    cid_found = {}
    #For each list of match at the specific threshold score, we search through the list and identify false positives based
    #off of four potential cases.
    for j in range(len(matches)):
        #Case one: the records are a match and this is the first time we have found this record combination, in which
        #song name we decide on which record we are going to consolidate the duplicate names
        if (data.iloc[matches[j][0]]['CID'] ==data.iloc[matches[j][1]]['CID']) and (data.iloc[matches[j][0]]['CID'] not in list(cid_found.keys())):
            cid_found[data.iloc[matches[j][0]]['CID']] =str(data.iloc[matches[j][1]]['all'])
            match_count= match_count+1
            dupes_found.append(data.iloc[matches[j][0]]['all'])
        #Case two: the records are a match and we already know which record we are standardizing the song to.
        elif (data.iloc[matches[j][0]]['CID'] ==data.iloc[matches[j][1]]['CID']) and (cid_found[data.iloc[matches[j][0]]['CID']]==str(data.iloc[matches[j][1]]['all'])) and data.iloc[matches[j][0]]['all'] not in dupes_found:
            dupes_found.append(data.iloc[matches[j][0]]['all'])
            
            match_count=match_count+1
        #Case four: False positive
        elif  (data.iloc[matches[j][0]]['CID'] !=data.iloc[matches[j][1]]['CID']) and data.iloc[matches[j][0]]['TID'] not in errors_found:
            errors_found.append(data.iloc[matches[j][0]]['TID'])
            error_count = error_count+1
        #All other cases are duplicate match records which can be ignored.
        else:
            pass
    appen_l = [threshold_val, match_count, error_count]
    analysis_df = analysis_df.append([appen_l])
    print('A threshold value of {} yields {} positives and false positives {}.'.format(threshold_val,match_count, error_count))

analysis_df.columns = ['Threshold Value', 'Match Count', 'Error Count']

In [ ]:
#Find the true number of entities and compute the portion that we have found at each score threshold
total_true_ents = len(data['CID'].unique())
analysis_df['Portion of Duplicates Found']  =  analysis_df['Match Count']/(len(data)-total_true_ents)

### Visualization: The Tradeoff Between Score Threshold and Portion of Duplicates Found

In [ ]:
plt.plot(analysis_df['Threshold Value'], analysis_df['Match Count'])
plt.plot(analysis_df['Threshold Value'], analysis_df['Error Count'])
plt.title('Total Positives and False Positives by Threshold of Similarity Model')
plt.show()

### Visualization: Errors Versus Matches 

In [ ]:
plt.scatter(x=analysis_df['Error Count'], y=analysis_df['Match Count'], c=analysis_df['Threshold Value'])
plt.title('Errors versus Matches in Vectorized Model (Colored by Threshold of Similarity Model)')
plt.show()

## Bonus: Finding the Optimum Threshold Value

With the information that we have in the analysis_df, we can analyze the graphs above to determine the optimum threhold value for our TF-IDF vector.  The way in which you select that value is a more qualitative excercise and depends on your specific use case.  For this illustration, let's take a purely quantative approach and graph the derivative of the plot above.  The derivative will help us determine the optimum number of matches that you can get for the least about of errors

In [ ]:
data= {}
data['x'] = analysis_df['Match Count'].tolist()
data['y'] = analysis_df['Error Count'].tolist()
data['y_p'] = np.diff(data['y']) / np.diff(data['x'])
data['x_p'] = (np.array(data['x'])[:-1] + np.array(data['x'])[1:]) / 2
plt.figure(1)
plt.plot(data['x_p'], data['y_p'], 'b')
plt.vlines(7618, 0,5)
plt.show()

As shown at the verticle line in the graph above, it looks like the number of errors per matches increases slowly and then drops before increasing exponentially.  Therefore it seems like the optimum threshold value is just short of 8000 matches.  We can find the exact the exact number of matches at that point by looking at where Y prime drops slightly before taking off.

In [ ]:
plt.plot((data['x_p']/data['y_p']))
plt.show()

In [ ]:
data['y_p']

In [ ]:
np.argwhere(data['y_p']==.125)

In [ ]:
data['x'][9]

In [ ]:
analysis_df[analysis_df['Match Count']==7618]

Therefore, it seems like the optimum threshold value for our entity resolution approach is .58 where we find over 81% of all duplicate records while only making 561 errors.